In [ ]:
%pip install -r PredictionModel/requirements.txt

In [ ]:
#!python PredictionModel/runPredModel.py patient-data/p001/p001.jpg
from PredictionModel.runPredModel import scanallxrays
import os
import pprint

pdir = "patient-data"
for p in os.listdir(pdir):
    patient_dir = os.path.join(pdir, p)
    print(f"patient {p}: {patient_dir}")
    pprint.pprint(scanallxrays(patient_dir))

In [1]:
from PredictionModel.convertDcm import convertall

convertall('patient-data/p001')

In [ ]:
imgfront = 'patient-data/p001/p001.jpg'
imglat = 'patient-data/p001/p001_2.jpg'
modelpath = 'NlpModel/nlpcheckpoint'

In [1]:
!python NlpModel/demo.py --img_frontal_path='patient-data/p001/p001.jpg' --img_lateral_path='patient-data/p001/p001_2.jpg' --model_path='NlpModel/nlpcheckpoint'

Traceback (most recent call last):
  File "c:\Users\mohaansh\Desktop\Capstone\Repo\chest-x-ray-ai\src\backend\detection-processor\NlpModel\demo.py", line 1, in <module>
    import tensorflow as tf
  File "C:\Users\mohaansh\AppData\Roaming\Python\Python311\site-packages\tensorflow\__init__.py", line 37, in <module>
    from tensorflow.python.tools import module_util as _module_util
ModuleNotFoundError: No module named 'tensorflow.python'
